In [ ]:
# primer: generiraj vse povezane grafe z n=6
n = 6
vsi_grafi = [G for G in graphs(n) if G.is_connected()]
print("Število povezanih grafov:", len(vsi_grafi))

def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

k = 2
grafi_k = [G for G in vsi_grafi if cyclomatic_number(G) == k]
print("Število grafov z µ(G)=2:", len(grafi_k))

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for neighbor in G.neighbors(start):
        if neighbor not in visited:
            newpaths = all_simple_paths(G, neighbor, end, visited)
            for p in newpaths:
                paths.append(p)
    return paths

def subpath_number(G):
    n = 0
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i, len(V)):
            n += len(all_simple_paths(G, V[i], V[j]))
    return n

for G in grafi_k:
    print(f"|V|={G.order()}, |E|={G.size()}, µ(G)={cyclomatic_number(G)}, p_n(G)={subpath_number(G)}")


In [ ]:
# === Projekt: Subpath number pri grafih z danim ciklomatskim številom ===

import itertools
from sage.all import *
import pandas as pd


# -------------------------------
# Funkcija za ciklomatsko število
# -------------------------------
def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

# -------------------------------
# Funkcija za štetje vseh enostavnih poti
# -------------------------------
def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for neighbor in G.neighbors(start):
        if neighbor not in visited:
            newpaths = all_simple_paths(G, neighbor, end, visited)
            for p in newpaths:
                paths.append(p)
    return paths

def subpath_number(G):
    """Vrne skupno število enostavnih poti (vključno z dolžino 0)."""
    V = list(G.vertices())
    n_paths = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            n_paths += len(all_simple_paths(G, V[i], V[j]))
    return n_paths

# -------------------------------
# Glavni izračun
# -------------------------------
rezultati = []  # seznam (n, µ, subpath_number)

# Za n = 3 do 7 (manjše vrednosti, da se ne zatakne)
for n in range(3, 10):
    print(f"\nObdelujem vse povezane grafe z n = {n} vozlišči ...")
    for G in graphs(n):
        if not G.is_connected():
            continue
        mu = cyclomatic_number(G)
        pn = subpath_number(G)
        rezultati.append((n, mu, pn))

print("\n--- Izračun končan ---")

# Pretvori rezultate v Sage DataFrame
df = pd.DataFrame(rezultati, columns=["n", "µ(G)", "p_n(G)"])


# Izračunaj minimalni in maksimalni subpath number za vsako kombinacijo (n, µ)
tabela = (
    df.groupby(["n", "µ(G)"])
    .agg(min_pn=("p_n(G)", "min"), max_pn=("p_n(G)", "max"), count=("p_n(G)", "count"))
    .reset_index()
)

# Izpiši tabelo rezultatov
print("\nTabela ekstremov (min / max subpath number):")
show(tabela)

# Shranimo tabelo kot CSV
tabela.to_csv("rezultati_subpath.csv", index=False)
print("\nRezultati shranjeni v datoteko 'rezultati_subpath.csv'.")

In [ ]:
from sage.all import *
import pandas as pd
import os

def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]

    if start == end:
        return [visited]

    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            for p in all_simple_paths(G, nei, end, visited):
                paths.append(p)
    return paths

def subpath_number(G):
    V = list(G.vertices())
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total


# ---------------- CSV INIT ----------------
CSV_FILE = "rezultati_subpath_live.csv"

if not os.path.exists(CSV_FILE):
    pd.DataFrame(columns=["n","mu","min_pn","min_g6","max_pn","max_g6"])\
        .to_csv(CSV_FILE, index=False)


def save_live(n, mu, min_pn, min_g6, max_pn, max_g6):
    df = pd.read_csv(CSV_FILE)

    df = df[~((df["n"] == n) & (df["mu"] == mu))]

    df.loc[len(df)] = [
        int(n), int(mu),
        min_pn, min_g6,
        max_pn, max_g6
    ]

    df.to_csv(CSV_FILE, index=False)


# ---------------- GLAVNI PROGRAM ----------------
for n in range(3, 10):

    print(f"\n>>> ZAČENJAM n = {n}")

    # 1) ZBEREMO GRAFE PO µ
    bucket = {}
    for G in graphs(n):
        if not G.is_connected():
            continue
        mu = cyclomatic_number(G)
        bucket.setdefault(mu, []).append(G)

    # 2) OBRAVNAVAMO µ PO VRSTI — takojšnja obdelava + zapis
    for mu in sorted(bucket.keys()):
        print(f"\n*** OBRAVNAVAM µ = {mu} (n={n}) ***")

        best_min = None
        best_max = None

        for G in bucket[mu]:
            pn = subpath_number(G)
            g6 = G.graph6_string()

            # MIN
            if best_min is None or pn < best_min[0]:
                best_min = (pn, g6, G.copy())

            # MAX
            if best_max is None or pn > best_max[0]:
                best_max = (pn, g6, G.copy())

        # PO ZAKLJUČKU µ — takojšnji zapis!
        min_pn, min_g6, Gmin = best_min
        max_pn, max_g6, Gmax = best_max

        print(f"  MIN={min_pn}, MAX={max_pn}")
        Gmin.show()
        Gmax.show()

        save_live(n, mu, min_pn, min_g6, max_pn, max_g6)



In [ ]:
# === Projekt: Subpath number pri grafih z danim ciklomatskim številom ===
# Delo v CoCalc-u (SageMath kernel)

import itertools
from sage.all import *
import pandas as pd


# -------------------------------
# Funkcija za ciklomatsko število
# -------------------------------
def cyclomatic_number(G):
    """Vrne ciklomatsko število grafa G."""
    return G.size() - G.order() + G.connected_components_number()


# -------------------------------
# Funkcija za štetje vseh enostavnih poti
# -------------------------------
def all_simple_paths(G, start, end, visited=None):
    """Rekurzivno poišče vse enostavne poti med start in end."""
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for neighbor in G.neighbors(start):
        if neighbor not in visited:
            newpaths = all_simple_paths(G, neighbor, end, visited)
            for p in newpaths:
                paths.append(p)
    return paths


def subpath_number(G):
    """Vrne skupno število enostavnih poti (vključno z dolžino 0)."""
    V = list(G.vertices())
    n_paths = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            n_paths += len(all_simple_paths(G, V[i], V[j]))
    return n_paths


# -------------------------------
# Glavni izračun
# -------------------------------
rezultati = []  # seznam (n, µ, subpath_number)

# Za n = 3 do 8
for n in range(8, 10):
    print(f"\nObdelujem grafe z n = {n} vozlišči ...")

    max_edges = n * (n - 1) // 2  # maksimalno število povezav

    # Za vsa možna ciklomatska števila µ
    for mu in range(0, max_edges - n + 2):
        m = mu + n - 1  # izpeljano iz µ = m - n + 1

        if m > max_edges:
            continue

        print(f"  → µ = {mu}, m = {m}")

        # Generiramo vse neizomorfne grafe z n vozlišči in m povezavami
        for G in graphs.nauty_geng(f"{n} {m}:{m}"):
            if not G.is_connected():
                continue

            pn = subpath_number(G)
            rezultati.append((n, mu, pn))

print("\n--- Izračun končan ---")

# Pretvori rezultate v DataFrame
df = pd.DataFrame(rezultati, columns=["n", "µ(G)", "p_n(G)"])

# Izračunaj minimalni in maksimalni subpath number za vsako kombinacijo (n, µ)
tabela = (
    df.groupby(["n", "µ(G)"])
    .agg(min_pn=("p_n(G)", "min"), max_pn=("p_n(G)", "max"), count=("p_n(G)", "count"))
    .reset_index()
)

# Izpiši tabelo rezultatov
print("\nTabela1 ekstremov (min / max subpath number):")
show(tabela)

# Shranimo tabelo kot CSV
tabela.to_csv("rezultati_subpath.csv", index=False)
print("\nRezultati shranjeni v datoteko 'rezultati_subpath1.csv'.")


In [ ]:
import itertools
from sage.all import *
import pandas as pd
import os

# --------------------------------------------------
# Funkcija za ciklomatsko število
# --------------------------------------------------
def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

# --------------------------------------------------
# Funkcija za štetje enostavnih poti
# --------------------------------------------------
def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            newp = all_simple_paths(G, nei, end, visited)
            paths.extend(newp)
    return paths

def subpath_number(G):
    V = G.vertices()
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total


# --------------------------------------------------
# Glavni izračun – z enoličnim zapisom grafov (graph6)
# --------------------------------------------------

os.makedirs("slike_min", exist_ok=True)
os.makedirs("slike_max", exist_ok=True)

rezultat_vrstice = []   # za CSV

for n in range(3, 9):
    print(f"\nObdelujem vse povezane grafe z n = {n} vozlišči ...")

    po_mu = {}   # mu : list of (pn, G)

    for G in graphs(n):
        if not G.is_connected():
            continue

        mu = cyclomatic_number(G)
        pn = subpath_number(G)

        if mu not in po_mu:
            po_mu[mu] = []
        po_mu[mu].append((pn, G))

    # min/max
    for mu in sorted(po_mu.keys()):
        seznam = sorted(po_mu[mu], key=lambda x: x[0])

        min_pn, G_min = seznam[0]
        max_pn, G_max = seznam[-1]

        # --- slike ---
        min_slika = f"slike_min/graf_min_n{n}_mu{mu}.png"
        max_slika = f"slike_max/graf_max_n{n}_mu{mu}.png"

        G_min.plot().save(min_slika)
        G_max.plot().save(max_slika)

        # --- enoličen zapis grafa (graph6) ---
        g6_min = G_min.graph6_string()
        g6_max = G_max.graph6_string()

        # --- zapišemo v tabelo ---
        rezultat_vrstice.append([n, mu, min_pn, max_pn,
                                 min_slika, max_slika,
                                 g6_min, g6_max])

print("\n--- Izračun končan ---")


# --------------------------------------------------
# CSV izpis
# --------------------------------------------------

df = pd.DataFrame(
    rezultat_vrstice,
    columns=[
        "n",
        "µ(G)",
        "min p_n(G)",
        "max p_n(G)",
        "slika_min",
        "slika_max",
        "graph6_min",
        "graph6_max"
    ]
)

df.to_csv("rezultati_subpath_grafi_enolicni.csv", index=False)


In [ ]:
# risanje konkretnih grafov

g6 = "F~~~w"       # tvoj graph6 niz
G = Graph(g6)     # rekonstruira graf
G.show()  